Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [31]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime

import glob
import time

In [32]:
# Cargo archivo
fecha = datetime.now().strftime("%Y%m%d")
#fecha = "20250117"
product = 'iphone'
carpeta_origen = "../data/3_feature_engineering"
carpeta_destino = "../data/4_download_description"

In [33]:
"""
Este script funciona así:
origen  - 03_feature_engineering
destino - 04_download_description

get_description                      -> itera por cada uno de los archivos de origen
    get_series_of_id_to_scrap        -> obtiene el archivo de origen
        get_old_download_description -> obtiene el archivo de destino /si lo hubiere
scrap_comments                       -> descarga los comentarios
"""


'\nEste script funciona así:\norigen  - 03_feature_engineering\ndestino - 04_download_description\n\nget_description                      -> itera por cada uno de los archivos de origen\n    get_series_of_id_to_scrap        -> obtiene el archivo de origen\n        get_old_download_description -> obtiene el archivo de destino /si lo hubiere\nscrap_comments                       -> descarga los comentarios\n'

In [34]:
#########
# UTILS #
#########
# Funciona!
def get_missing_records_from_csv(old_download_description, last_feature_engineering):
    """ Devuelve una serie con los id de los registros 
    que están en el segundo archivo pero no en el primero.
    Es decir, con los ids a scrappear
    """
    print(f"Comparando los archivos:\n- {old_download_description}\n- {last_feature_engineering}")
    df1 = pd.read_csv(old_download_description)
    df2 = pd.read_csv(last_feature_engineering)
    df_merged = df2.merge(df1, on='id', how='left', indicator=True)
    missing_records = df_merged[df_merged['_merge'] == 'left_only']
    return missing_records['id']
def get_old_download_description(path_product_data_n):
    """
    Busca el archivo de download_description del scrap anterior de ese dia 
    y si no del anterior
    si 
    path_product_data_n = '../data/3_feature_engineering/iphone_20250225_1.csv'
    return ../data/4_download_description/iphone_20250225_0.csv 
    siempre que iphone_20250225_0 existe. También buscaria el dia de antes
    """
    import datetime
    base_name = os.path.basename(path_product_data_n)
    date_str = base_name.split('_')[1]
    n = base_name.split('_')[-1].split('.')[0]
    if n != '0':
        old_n = str(int(n) - 1)
        old_download_description = f"{carpeta_destino}/iphone_{date_str}_{old_n}.csv"
    else:
        date_obj = datetime.datetime.strptime(date_str, '%Y%m%d')
        previous_day = date_obj - datetime.timedelta(days=1)
        previous_day_str = previous_day.strftime('%Y%m%d')
        pattern = f"{carpeta_destino}/iphone_{previous_day_str}_*.csv"
        files = glob.glob(pattern)
        if files:
            max_n = max([int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in files])
            old_download_description = f"{carpeta_destino}/iphone_{previous_day_str}_{max_n}.csv"
        else:
            old_download_description = None
    if not old_download_description:
        print(f"No se encontraron archivos para el día anterior {previous_day_str}")
    return old_download_description

# Ejemplo de uso
# last_feature_engineering = '../data/3_feature_engineering/iphone_20250225_1.csv'
def get_series_of_id_to_scrap(last_feature_engineering):
    old_download_description = get_old_download_description(last_feature_engineering)
    print("old_download_description", old_download_description)
    if old_download_description is not None:
        return(get_missing_records_from_csv(old_download_description, last_feature_engineering))
    else:
        return None

In [ ]:
# 263 anuncios - 25 min. con sleep de 1
# 5s por anuncio si va lento. Si va bien 1.5s
# Escrapeo objetos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario

def scrap_comments(df):
    """
    Dado un df con una columna 'url_articulo' de wallapop:
    accede a la url_articulo, 
    busca el comentario y
    lo guarda en la columna 'comentario'
    """
    df_len = len(df)
    df.loc[:, 'comentario'] = None  # modificacion para quitar un warning. Antes: df['comentario'] = None
    if 'url_articulo' not in df.columns:
        raise ValueError("Error: La columna 'url_articulo' no existe en el DataFrame.")
    print("Tiempo estimado de inicio: 25s...")
    # setup del driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--enable-unsafe-webgl')
    options.add_argument('--disable-usb')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    driver.get(df.loc[0]['url_articulo']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()  # Quita cookies y carga primera web
    for index in range(len(df)):
        start_time = time.time()  # Marca de tiempo al inicio de la iteración
        print(index, '/', df_len)
        driver.get(df.loc[index]['url_articulo']) 
        try:
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url_articulo'], ': \n', elemento_comentario)
        except:
            #element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            df.loc[index, 'comentario'] = "not found"
            print(df.loc[index]['url_articulo'], df.loc[index, 'comentario'])

        end_time = time.time()  # Marca de tiempo al final de la iteración
        execution_time = end_time - start_time  # Calcula el tiempo de ejecución
        print(f"Estimated time to finish: {((df_len - index) * execution_time/60):.2f}min")
        print(f"{execution_time:.2f}s")  
    driver.quit()
    return df

In [ ]:
def get_description(n):
    """ download the description of products that doesnt satisfied the aimed condition"""
    n = str(n)
    last_feature_engineering = os.path.join(carpeta_origen, f"{product}_{fecha}_{n}.csv")
    if not os.path.exists(last_feature_engineering):
        return 
    ids_to_scrap = get_series_of_id_to_scrap(last_feature_engineering)
    df_original = pd.read_csv(last_feature_engineering)
    if ids_to_scrap is None:
        # si no hay csv anteriores para comparar escanea todos
        df_filtered = df_original
    else:
        # si hay csv anteriores filtra los que ya tienen comentarios
        df_filtered = df_original[df_original['id'].isin(ids_to_scrap)]

    # df_original = pd.read_csv(filepath) ########### esto funciona
    ###############################
    # Selecciono el sub-dataframe #
    ###############################
    # Selecciono:       # as_good_as_new 
    #                   # iphone 15. 
    #                   bateria = NaN
    # estado = new...
    df_to_scrap = df_filtered[#(df_filtered['estado'] == estado) & 
                            #(df_filtered['gen'] == iphone) & 
                            (pd.isna(df_filtered['bateria']))]
    df_to_scrap.reset_index(drop=True, inplace = True)
    df_with_comments = scrap_comments(df_to_scrap)
    ##########################
   
    subdf_with_comments = df_with_comments[['id', 'comentario']]
    df_filtered = pd.merge(df_filtered, subdf_with_comments, on='id', how='left')
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    df_filtered = df_filtered.reset_index(drop=True) # sin indice
    df_filtered.to_csv(os.path.join(carpeta_destino, f"{product}_{fecha}_{n}.csv"))

for n in range(10):
    get_description(n)

No se encontraron archivos para el día anterior 20250224
old_download_description None
Tiempo estimado de inicio: 25s...


C:\Users\ACER\AppData\Local\Temp\ipykernel_6932\3278886921.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comentario'] = None


0 / 90
https://es.wallapop.com/item/iphone-16-plus-azul-1105887001 : 
 Detalles del producto
Se vende IPhone 16 plus azul con factura y cable. Esta como nuevo. Se entrega en mano en Madrid.
Estado
Como nuevo
Marca
Apple
Modelo
Iphone 16 plus
Color
Azul
Capacidad de almacenamiento
128 GB
Estimated time to finish: 7.68min
5.12s
1 / 90
https://es.wallapop.com/item/iphone-16-pro-256-1101459978 : 
 Detalles del producto
comprado en sept
usado 2 meses xq me compre el max
Estado
Como nuevo
Marca
Apple
Modelo
Iphone 16 pro
Color
Negro
Capacidad de almacenamiento
256 GB
Estimated time to finish: 7.33min
4.94s
2 / 90
https://es.wallapop.com/item/iphone-16-pro-max-256gb-1105875789 : 
 Detalles del producto
Iphone 16 pro max:

capacidad: 256GB 
Bateria: 100%
comprado: Septiembre 2024
detalles: impoluto sin detalles.

lo vendo por motivos personales, tiene garantía y esta comprado directamente en la apple, tengo factura y caja. NO SE REGALA, NO REVENDEDORES, NO SE CAMBIA POR NADA, SOLO DINERO
Estad

In [37]:
# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441